## Neighborhoods in Toronto | Part 1  

### Scraping Wiki Page to create a dataframe

In [39]:
#importing libraries & installing geopy
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import pandas as pd
import geocoder
import requests
from bs4 import BeautifulSoup
print("Imported!")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\emhas\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  conda                                4.9.2-py38haa244fe_0 --> 4.10.1-py38haa244fe_0




geopy-2.1.0          | 64 KB   

In [27]:
source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df1=pd.DataFrame(table_contents)
df1['Borough']=df1['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Neighborhoods in Toronto | Part 2

### Using the Geocoder package to get the latitude and the longitude coordinates of each neighborhood ? adding to the dataframe

In [15]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [31]:
# gecoder did not work as expected, so I used the data file from the course
df2= pd.read_csv("./Geospatial_Coordinates.csv")
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
#rename column
df2= df2.rename(columns={'Postal Code': 'PostalCode'})
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
# merging two data frames
df3 = pd.merge(df1, df2, how='left', on='PostalCode')
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Explore and cluster the neighborhoods in Toronto

### Use geopy library to get the latitude and longitude values of Toronto

In [40]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [43]:
#installing folium map
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.10.14-0
  - defaults/win-64::ca-certificates-2020.10.14-0done

## Package Plan ##

  environment location: C:\Users\emhas\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1h             |       he774522_0         5.8 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           To

In [45]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(
        df3['Latitude'], 
        df3['Longitude'], 
        df3['Borough'], 
        df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

In [47]:
CLIENT_ID = '45B11BRVQDE0KV1BLV04O3CYCTYO0X524EXMHOUQH3ZL20XB' # your Foursquare ID
CLIENT_SECRET = 'ONW33JN0KW5YEUVX3OV2XSFODKDKJSA0FTMGVZQXEUHPUVJJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 45B11BRVQDE0KV1BLV04O3CYCTYO0X524EXMHOUQH3ZL20XB
CLIENT_SECRET:ONW33JN0KW5YEUVX3OV2XSFODKDKJSA0FTMGVZQXEUHPUVJJ


In [65]:
# Get the neighborhood's name.
df3.loc[99, 'Neighborhood']

'Church and Wellesley'

In [66]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = df3.loc[99, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[99, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[99, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Church and Wellesley are 43.6658599, -79.38315990000001.


In [67]:
# getting the top 50 venues that are in Church and Wellesley within a radius of 500 meters.
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    500, 
    50)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=45B11BRVQDE0KV1BLV04O3CYCTYO0X524EXMHOUQH3ZL20XB&client_secret=ONW33JN0KW5YEUVX3OV2XSFODKDKJSA0FTMGVZQXEUHPUVJJ&v=20180605&ll=43.6658599,-79.38315990000001&radius=500&limit=50'

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ce748b8175de04dbf62f27'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Village',
  'headerFullLocation': 'The Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 43.6703599045, 'lng': -79.37695070062593},
   'sw': {'lat': 43.6613598955, 'lng': -79.3893690993741}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51a11515498ef97667980b8c',
       'name': 'DanceLifeX Centre',
       'location': {'address': '619 Yonge Street',
        'crossStreet': 'btwn Isabella St & Gloucester St',
        'lat': 43.66695583448771,
        'lng': -79.3852972984314,
        'labeledLa

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [73]:
#structure everything in the dataframe after data cleaning
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,DanceLifeX Centre,Dance Studio,43.666956,-79.385297
1,Fabarnak,Restaurant,43.666377,-79.380964
2,Storm Crow Manor,Theme Restaurant,43.666840,-79.381593
3,Smith,Breakfast Spot,43.666927,-79.381421
4,Como En Casa,Mexican Restaurant,43.665160,-79.384796


In [74]:
# check how many venues were returned 
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.
